<a href="https://colab.research.google.com/github/moseskim/bert_nlp/blob/main/section_2/01_pytorch_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch 기초

PyTorch 기초를 학습하기 위해 신경망을 훈련시켜 손으로 쓴 문자를 인식해봅니다.


## 데이터 로딩

`torchvision.datasets`를 사용해 손으로 쓴 문자 데이터를 로딩하고, **DataLoader**를 설정합니다.  
`DataLoader`를 사용하면 데이터 로딩이나 미니 배치 알고리즘 구현을 매우 쉽게 할 수 있습니다.  
`torchvision.datasets`에는 MNIST 외에도 다양한 데이터셋이 준비되어 있습니다.  
https://pytorch.org/docs/stable/torchvision/datasets.html

In [ ]:
import torch
from torchvision.datasets import MNIST  # 손으로 쓴 문자 이미지 데이터
from torchvision import transforms
from torch.utils.data import DataLoader

# 훈련 데이터 취득
mnist_train = MNIST("./data",
                    train=True, download=True,
                    transform=transforms.ToTensor())
# 테스트 데이터 취득
mnist_test = MNIST("./data",
                   train=False, download=True,
                   transform=transforms.ToTensor())
print("훈련 데이터 수: ", len(mnist_train), "테스트 데이터 수: ", len(mnist_test))

# DataLoader 설정
img_size = 28
batch_size = 256
train_loader = DataLoader(mnist_train,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(mnist_test,
                         batch_size=batch_size,
                         shuffle=False)

손으로 쓴 문자의 이미지 크기는 28x28이 됩니다.



## 모델 구축

여기에서는 `nn.Module` 모듈을 상속한 클래스로 모델을 구축합니다.  
`.cuda()`에 의해 모델의 계산은 GPU 상에서 수행됩니다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(img_size*img_size, 1024)  # 전결합층
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, img_size*img_size)  # 배치 크기 x 입력 수
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.cuda()  # GPU 대응
print(net)

## 학습

모델을 훈련합니다.  
`DataLoader`를 사용해 미니 배치를 꺼내서 훈련 및 평가를 수행합니다.

1 에폭 안에서 여러 차례 미니 배치를 사용해 훈련이 수행되고, 미니 배치 알고리즘이 구현되게 됩니다.  
학습에는 시간이 걸리므로 수정 → 노트 설정 → 하드웨어 가속기에 GPU가 선택되어 있는 것을 확인합니다.


In [ ]:
from torch import optim

# 교차 엔트로비피 오차 함수
loss_fnc = nn.CrossEntropyLoss()

# SGD
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 손실 로그
record_loss_train = []
record_loss_test = []

# 학습
for i in range(10):  # 10 에폭 학습
    net.train()  # 훈련 모드
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):  # 미니 배치(x, t)를 꺼낸다
        x, t = x.cuda(), t.cuda()  # GPU 대응
        y = net(x)
        loss = loss_fnc(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)

    net.eval()  # 평가 모드
    loss_test = 0
    for j, (x, t) in enumerate(test_loader):  # 미니 배치(x, t)를 꺼낸다
        x, t = x.cuda(), t.cuda()
        y = net(x)
        loss = loss_fnc(y, t)
        loss_test += loss.item()
    loss_test /= j+1
    record_loss_test.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## 오차 추이

훈련 데이터, 테스트 데이터로 오차 추이를 그래프로 표시합니다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## 정답율

모델의 성능을 파악하기 위해 테스트 데이터를 사용해 정답율을 측정합니다.

In [ ]:
correct = 0
total = 0
for i, (x, t) in enumerate(test_loader):
    x, t = x.cuda(), t.cuda()  # GPU 대응
    x = x.view(-1, img_size*img_size)
    y = net(x)
    correct += (y.argmax(1) == t).sum().item()
    total += len(x)
print("정답율: ", str(correct/total*100) + "%")